In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
Error adding Ethereum blockchain to database Ethereum, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(Ethereum) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'Ethereum', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) skipping...
Error adding exchange carbon_v1 to database, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "exchanges_pkey"
DETAIL:  Key (id)=(6) already exists.

[SQL: INSERT INTO exchanges (id, name, blockchain_name) VALUES (%(id)s, %(name)s, %(blockchain_name)s)]
[parameters: {'id': 6, 'name': 'carbon_v1', 'blockchain_name': 'Ethereum'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) ski

C:\Users\Kveen\AppData\Local\Temp\ipykernel_5148\1582963263.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on TENDERLY [NBTest022]

## Tenderly Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_TENDERLY)
assert C.DATABASE == C.DATABASE_POSTGRES
# assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_TENDERLY
assert C.PROVIDER == C.PROVIDER_TENDERLY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"




In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)


Number of curves: 3539
Number of tokens: 2230
Exchanges: {'uniswap_v3', 'bancor_v3', 'uniswap_v2', 'carbon_v1', 'bancor_v2', 'sushiswap_v2'}


In [4]:
used_curves = [
    # '340282366920938463463374607431768211527-0', #actually deleted from carbon
    # '340282366920938463463374607431768211527-1', #actually deleted from carbon
    # '2722258935367507707706996859454145691730-1',
    # '1701411834604692317316873037158841057360-1',
    # '1701411834604692317316873037158841057360-0',
    # '9187623906865338513511114400657741709390-0',
    # '12590447576074723148144860474975423823947-0',
    # '13951577043758477001998358904702496669779-1'
]
subcurves = [x for x in CCm if x.cid not in used_curves]
no_0x0 = [x for x in subcurves if '0x0-1AD5' not in x.pair]
CCm2 = CPCContainer([x for x in no_0x0 if x.params.exchange not in ['bancor_v2']])
len(CCm2)
#print(CCm2)

3442

In [5]:
pairs = [x.pair.split('/') for x in CCm if x.params.exchange == 'carbon_v1']
len(set([x for itm in pairs for x in itm]))

27

In [6]:
pairs = [x.pair for x in CCm if x.params.exchange == 'carbon_v1']
len(set(pairs))

61

In [7]:
# CCm.bycid("5784800237655953878877368326340059594782").p

In [8]:
# CCm.bycid("5d5cebf33c0049258c40db6bc90b1558").p

In [9]:
# CCm.bycid("9a52c5bdef1c402497d1c4f5d7501f61").p

In [10]:
# CCm.bycid("12590447576074723148144860474975423823947-0").p

In [11]:
# 1/CCm.bycid("12590447576074723148144860474975423823947-1").p

## Actual run

In [12]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [13]:
ops = bot._find_arbitrage_opportunities_carbon_multi_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ARBOPPS)

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeW

In [14]:
ops

(Decimal('68990.9453488580655589602057020594968088019626915527449106946278334362432360649'),
                                                USDC-eB48  WETH-6Cc2
 50ac5ace09c1483987af46c60c551073            8.702803e+04 -49.626459
 1701411834604692317316873037158841057353-0 -7.851120e+00   0.004235
 1701411834604692317316873037158841057376-0 -2.900000e+04  10.357143
 1701411834604692317316873037158841057378-0 -1.500000e+04   3.750000
 1701411834604692317316873037158841057377-0 -1.800000e+04   5.806452
 1701411834604692317316873037158841057375-0 -2.500000e+04  11.904762
 1701411834604692317316873037158841057292-0 -6.141325e+00   0.003317
 1701411834604692317316873037158841057339-0 -4.165539e-01   0.000234
 1701411834604692317316873037158841057296-0 -1.994537e+00   0.001033
 1701411834604692317316873037158841057360-0 -1.162787e+01   0.004948
 AMMIn                                       8.702803e+04  31.832123
 AMMOut                                     -8.702803e+04 -49.626459
 TOTAL NET

In [15]:
# ord = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ORDSCAL)
# ord

In [16]:
# ti = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_TI)
# ti

In [17]:
# ti

In [18]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [19]:
# candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
# candf.reset_index(inplace=True, drop=True)
# candf.iloc[5][1]

In [20]:
# candf.iloc[0][0]

In [21]:
aggregated = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_AGGTI)
#aggregated

for trade in aggregated:
    print(trade.amtin, trade.tknin, trade.amtout, trade.tknout)


C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeW

31.800291166209057 WETH-6Cc2 -87028.03140485166 USDC-eB48
87028.03140489757 USDC-eB48 -49.626459348251956 WETH-6Cc2


In [22]:
exact = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_EXACT)
# exact

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeW

In [23]:
for trade in exact:
    print(trade.amtin, trade.tknin, trade.amtout, trade.tknout)

31.800291166209058106 WETH-6Cc2 86758.444664 USDC-eB48
86758.444664 USDC-eB48 49.319757603950395836 WETH-6Cc2


In [24]:
# assert False

In [25]:
# agg = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_AGGTI)
# agg

In [26]:
# agg[0].raw_txs

In [27]:
# agg[1].raw_txs

In [28]:
exact[0].amtin_wei, exact[0].amtout_wei

(31800291166209058106, 86758444664)

In [29]:
exact[1].amtin_wei, exact[1].amtout_wei

(86758444664, 49319757603950395836)

In [30]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeW

In [31]:
route_struct, flashloan_amount, flashloan_token_address

([{'exchangeId': 6,
   'targetToken': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'minTargetAmount': 1,
   'deadline': 1784168320,
   'customAddress': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'customInt': 2000,
   'customData': '0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000090000000000000000000000000000000500000000000000000000000000000049000000000000000000000000000000000000000000000000000f07948cdd07aa00000000000000000000000000000005000000000000000000000000000000600000000000000000000000000000000000000000000000008f972ac81ed1b000000000000000000000000000000000050000000000000000000000000000006200000000000000000000000000000000000000000000000033fd5a86c9cea00000000000000000000000000000000005000000000000000000000000000000610000000000000000000000000000000000000000000000005080080a1e9ec400000000000000000000000000000000050000000000000000000000000000005f000000000000000000000000000000000000000000000000a50b

# Tenderly Tx Submission

In [32]:
# C.w3.provider.endpoint_uri

In [33]:
# C.w3.eth.getBlock(C.w3.eth.block_number).timestamp

In [34]:
# C.provider.CARBON_CONTROLLER_CONTRACT.strategy(13951577043758477001998358904702496669788)

In [35]:
### This to manually submit
bot._validate_and_submit_transaction_tenderly(
        ConfigObj = C,
        route_struct = route_struct,
        src_amount = flashloan_amount,
        src_address = flashloan_token_address,
            )

AttributeDict({'transactionHash': HexBytes('0x57977fd43552a4483e3b0ef88635cc6d268791a2b2329ec5560f2a25988ff78d'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x09eb6d237da0eb2a5fbdbd18067d51d32b60ce2cf330a4c723339406f8f6eaa4'),
 'blockNumber': 17213358,
 'from': '0x28C6c06298d514Db089934071355E5743bf21d60',
 'to': '0x41Eeba3355d7D6FF628B7982F3F9D055c39488cB',
 'cumulativeGasUsed': 690267,
 'gasUsed': 690267,
 'effectiveGasPrice': 0,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'address': '0x649765821D9f64198c905eC0B2B037a4a52Bc373',
   'topics': [HexBytes('0xc322efa58c9cb2c39cfffdac61d35c8643f5cbf13c6a7d0034de2cf18923aff3'),
    HexBytes('0x000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
    HexBytes('0x000000000000000000000000eef417e1d5cc832e619ae18d2f140de2999dd4fb'),
    HexBytes('0x00000000000000000000000041eeba3355d7d6ff628b7982f3f9d055c39488cb')],
   'data': '0x000000000000000000000000000000000000000000000001b95154549be2253a',
   'b

# MAINNNET Preferred Tx Submission Method

In [36]:
# from fastlane_bot.helpers import TxHelpers

In [37]:
# pool = (
#     bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
# )
# bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# # Init TxHelpers
# tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# # Submit tx



In [38]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

# MAINNET Forced Tx Submission Method

In [39]:
# # Initialize tx helper
# tx_helper = TxHelper(
#     usd_gas_limit=55,
#     ConfigObj=bot.ConfigObj,
#     gas_price_multiplier = 1
# )
# # # Submit the transaction
# # tx_helper.submit_flashloan_arb_tx(
# #     arb_data=route_struct,
# #     flashloan_token_address=flashloan_token_address,
# #     flashloan_amount=flashloan_amount,
# #     verbose=True,
# # )

In [40]:
# tx_helper.gas_limit